

#Description: this is a programm to check if an email is spam(1) or not(0)

In [22]:
#importing modules
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

In [23]:
# load the data
from google.colab import files
uploaded = files.upload()


Saving spam.csv to spam (1).csv


In [24]:
# read and print
df= pd.read_csv("spam.csv")
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
# print the shape(number and columns of data set)
df.shape
df.columns

Index(['Category', 'Message'], dtype='object')

In [26]:
# check for duplicates and delete it
df.drop_duplicates(inplace=True)

In [27]:
# lets see new shape i.ie numbers and columns now
df.shape
# we didnt had any to begin with

(5157, 2)

In [28]:
# show the number of missing data for each column
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [29]:
# download the stopword pakage
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
# peocessing words:
def process_text(text):

  # 1 remove punctuations
  no_punc=[char for char in text if char not in string.punctuation]
  no_punc=''.join(no_punc)

  # 2 remove stopwords
  clean_words=[word for word in no_punc.split() if word.lower() not in stopwords.words("english")]

  # 3 return a list of clean words
  return clean_words

In [32]:
# tokenization (also called lemmas. its a lists if tokens)
# here all the words are seperated by commas
df['Message'].head().apply(process_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Message, dtype: object

In [ ]:
# Example on how we are going to process this data to form predictions(deletable)
ex_message1='nice fuck bitch nice' 
ex_message2='bitch you fuck you you you fuck'
print(ex_message1)
print()

# convert text into matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
bow1=CountVectorizer(analyzer= process_text).fit_transform([[ex_message1],[ex_message2]]) #bow=bag of words
print(bow1)
print()
print(bow1.shape)


In [34]:
# Convert the data into matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
messages_bow=CountVectorizer(analyzer= process_text).fit_transform(df['Message']) #bow=bag of words


In [35]:
# now we will split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(messages_bow, df['Category'], test_size=0.20, random_state=0)
# feature trainig and testing set # target training and testing set

In [36]:
# shape of messages_bow
messages_bow.shape

(5157, 11420)

In [37]:
# creating and training Naive Bayes Classifier model. we are using multinomial NBC which is used for classification with discreet features 
from sklearn.naive_bayes import MultinomialNB
classifier= MultinomialNB().fit(x_train, y_train)

In [38]:
# ḷets print the prdictions
print(classifier.predict(x_train))
# print the actual values
print(y_train.values)

[0 1 0 ... 0 0 0]
[0 1 0 ... 0 0 0]


In [39]:
# how good is our training model(evaluate)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred= classifier.predict(x_train)
print(classification_report(y_train, pred) )
print()
print("confusion matrix: \n ", confusion_matrix(y_train, pred))
print()
print('accuracy:', accuracy_score(y_train, pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3619
           1       0.98      0.97      0.98       506

    accuracy                           0.99      4125
   macro avg       0.99      0.99      0.99      4125
weighted avg       0.99      0.99      0.99      4125


confusion matrix: 
  [[3611    8]
 [  13  493]]

accuracy: 0.9949090909090909


In [40]:
# now lets do it with the test values
print(classifier.predict(x_test))
# print the actual values
print(y_test.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [41]:
# how good is our model works on testing dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred= classifier.predict(x_test)
print(classification_report(y_test, pred) )
print()
print("confusion matrix: \n ", confusion_matrix(y_test, pred))
print()
print('accuracy:', accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       897
           1       0.81      0.93      0.86       135

    accuracy                           0.96      1032
   macro avg       0.90      0.95      0.92      1032
weighted avg       0.96      0.96      0.96      1032


confusion matrix: 
  [[867  30]
 [ 10 125]]

accuracy: 0.9612403100775194


In [42]:
# end